In [15]:
!pip install -q kaggle

In [17]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ./kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [18]:
!kaggle datasets download -d nursyahrina/chat-sentiment-dataset
!unzip chat-sentiment-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/nursyahrina/chat-sentiment-dataset
License(s): copyright-authors
  0% 0.00/7.42k [00:00<?, ?B/s]
100% 7.42k/7.42k [00:00<00:00, 12.8MB/s]
Archive:  chat-sentiment-dataset.zip
  inflating: chat_dataset.csv        


In [1]:
import pandas as pd
import numpy as np

In [20]:
df1 = pd.read_csv('/content/chat_dataset.csv')
df1.head()

,message,sentiment
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative


In [26]:
data = np.zeros([100,4])
data = pd.DataFrame(data, columns=['ucid', 'chat', 'question', 'intent'])

In [27]:
ucid = [100, 200, 300, 400] * 25
ucid.sort()
ucid = np.array(ucid)

chat = df1.message.iloc[:80]
chat = np.append(chat, np.repeat(np.nan, 20))
np.random.shuffle(chat)

intent = ['Yes - Chit Chat', 'Yes - Review Medical', 'Yes - background Info', 'No - Background Info'] * 25
intent = np.array(intent)
np.random.shuffle(intent)

question = df1.message.iloc[100:130]
question = np.append(question, np.repeat(np.nan, 70))
np.random.shuffle(question)

data.ucid = ucid
data.intent = intent
data.question = question
data.chat = chat
data

,ucid,chat,question,intent
0,100,This product is overpriced,NaN,No - Background Info
1,100,NaN,The museum was dull,No - Background Info
2,100,The traffic is horrible,NaN,Yes - Review Medical
3,100,This phone is amazing,NaN,Yes - Chit Chat
4,100,NaN,NaN,No - Background Info
...,...,...,...,...
95,400,NaN,The hotel was noisy during the day,Yes - Review Medical
96,400,This is a great product,NaN,Yes - background Info
97,400,I'm not impressed with this,The traffic is heavy,No - Background Info
98,400,I'm not satisfied with this product,NaN,Yes - Chit Chat


In [55]:
def test(data, ucid_number, rolling=0):
  data_temp = data[data.ucid == ucid_number]

  if rolling>0:
    final_chat = []
    final_question = []
    final_intent = []
    for i in range(data_temp.shape[0]):
      temp_chat = [' '.join(data_temp.iloc[i:i+rolling].chat.dropna().tolist())]
      final_chat.append(temp_chat)
      final_question.append(data_temp.question[i:i+rolling].dropna().tolist())
      final_intent.append(data_temp[~data_temp.intent.str.contains('Chit', regex=True, na=False)].loc[i:i+rolling].intent.unique().tolist())

  else:
    final_chat = [' '.join(data_temp.chat.dropna().tolist())]
    final_question = data_temp.question.dropna().tolist()
    final_intent = data_temp[~data_temp.intent.str.contains('Chit', regex=True, na=False)].intent.unique().tolist()

  return final_chat, final_question, final_intent

final_chat, final_question, final_intent = test(data, 100)
final_question

['The museum was dull',
 'I regret buying this',
 'This movie was disappointing',
 "I'm not happy with this product",
 "I'm ambivalent about this",
 'The hike was invigorating',
 "I don't really have an opinion on this"]